In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/MyDrive/CI for PD")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy import linalg as la
import datetime as dt
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm, trange
from numba import jit, njit
try:
    import multiprocess
except:
    import multiprocessing as multiprocess
import warnings
warnings.filterwarnings('ignore')

In [ ]:
initial_states = ["AAA", "AA", "A", "BBB", "BB", "B", "CCC", "Default"]

final_states = ["AAA", "AA", "A", "BBB", "BB", "B", "CCC", "Default", "NR"]

In [ ]:
sorted(os.listdir("Data/pds_ratings_reduced"))

['pds_2011_200_rating_reduced.csv',
 'pds_2012_200_rating_reduced.csv',
 'pds_2013_200_rating_reduced.csv',
 'pds_2014_200_rating_reduced.csv',
 'pds_2015_200_rating_reduced.csv',
 'pds_2016_200_rating_reduced.csv',
 'pds_2017_200_rating_reduced.csv',
 'pds_2018_200_rating_reduced.csv',
 'pds_2019_200_rating_reduced.csv',
 'pds_2020_200_rating_reduced.csv',
 'pds_2021_200_rating_reduced.csv']

In [ ]:
pds = [pd.read_csv(f"Data/pds_ratings_reduced/{f}") for f in sorted(os.listdir("Data/pds_ratings_reduced"))]

In [ ]:

pds_all = pd.concat([pds[i] for i in [0, 1,2, 3,4,5, 6,7]], axis=1)#pd.concat(pds, axis=1)

In [ ]:
pds_all

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000
4,0.000397,0.000327,0.000000,0.000371,0.000000,0.000325,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000
5,0.050446,0.047874,0.000627,0.043209,0.022624,0.043100,0.000542,0.052720,0.000413,0.021906,...,0.000000,0.000221,0.000000,NaN,0.000546,NaN,0.000000,0.000281,0.000000,0.000000
6,0.051348,0.000000,0.049084,0.015863,0.048358,0.017102,0.035706,0.031018,0.050047,0.104566,...,0.025843,0.022938,0.025902,NaN,0.057713,NaN,0.047854,0.029314,0.000000,0.026055
7,2.718282,2.718282,2.718282,2.718282,2.718282,2.718282,2.718282,2.718282,2.718282,2.718282,...,2.718282,2.718282,2.718282,NaN,2.718282,NaN,2.718282,2.718282,2.718282,2.718282
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000


In [ ]:
pds[0].dropna(axis=1).quantile(q=[0.025, 0.975], axis=1).T*10000

,0.025,0.975
0,0.000000,0.000000
1,0.000000,0.000000
2,0.000000,1827.992550
3,0.000000,0.034507
4,0.000000,5.198582
5,0.000000,683.250273
6,0.000000,665.508506
7,27182.818285,27182.818285
8,0.000000,0.000000


In [ ]:
(pds_all[pds_all.sum(axis=0)[pds_all.sum(axis=0)!=0].index.tolist()].dropna(axis=1).quantile(q=[0.025, 0.975], axis=1).T*10000)

,0.025,0.975
0,0.000000,0.000000
1,0.000000,0.000000
2,0.000000,1104.860408
3,0.000000,0.035362
4,0.000000,5.885677
5,0.000000,597.049747
6,0.000000,1017.503985
7,27182.818285,27182.818285
8,0.000000,0.000000


In [ ]:
non_param_ci = pd.concat([pds_all.dropna(axis=1).mean(axis=1), pds_all.dropna(axis=1).quantile(q=[0.025, 0.975], axis=1).T], axis=1)*10000
non_param_ci.columns = ["PD", "Lower", "Upper"]
non_param_ci.index = final_states
non_param_ci.drop(["Default", "NR"], axis=0, inplace=True)
non_param_ci

,PD,Lower,Upper
AAA,5.857229e-10,0.0,0.000000
AA,7.772137e-06,0.0,0.000000
A,4.381830e+01,0.0,1104.852943
BBB,3.364505e-03,0.0,0.035216
BB,7.728454e-01,0.0,5.876990
B,1.133678e+02,0.0,595.521518
CCC,3.306631e+02,0.0,1017.338979
